In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import csv
import time
import re
import urllib
import json
import numpy as np
import threading
from tqdm import tqdm
import pickle
import cloudscraper
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By
from selenium_stealth import stealth

In [2]:
with open(f"../use_link_load_data_0_1000000/link_data_stack_overflow_python_0_200000.pickle", "rb") as f:
    save_link = pickle.load(f)

In [4]:
max_pages = 24
count_thread = 8
per_vm = 8
vm_number = 0 + 10*0
num_per_thread = max_pages//count_thread
page_last = max_pages%count_thread
all_thread = [None]*count_thread
code_question = [None]*count_thread
code_answer = [None]*count_thread
webs = "https://stackoverflow.com/"
hours = 10
minute = 30

In [5]:
def save_code_question_answer_all_data(code_question,code_answer,start,stop,number_thread):
    try:
        st = time.time()
        count = 0
        code_answer_in_link = []
        code_question_in_link = []
        for i in save_link[start:stop]:
            url = "https://stackoverflow.com/" + i
            from selenium import webdriver 
            from selenium.webdriver.chrome.service import Service as ChromeService 
            from webdriver_manager.chrome import ChromeDriverManager 
            
            options = webdriver.ChromeOptions() #newly added 
            options.headless = True #newly added 
            while True:
                with webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options) as driver: #modified 
                    stealth(driver,
                    languages=["en-US", "en"],
                    vendor="Google Inc.",
                    platform="Win32",
                    webgl_vendor="Intel Inc.",
                    renderer="Intel Iris OpenGL Engine",
                    fix_hairline=True,
                    )
                    driver.get(url)
                    #print("Page URL:", driver.current_url) 
                    #print("Page Title:", driver.title)
                    answers = driver.find_elements(By.XPATH, "//div[@id='answers']//code")
                    questions = driver.find_elements(By.CLASS_NAME,'question-hyperlink')
                    #print(answers[0].text)
                    if (driver.title != "Just a moment...") and (len(questions) > 0) and (len(answers) > 0) :
                        answers_in_link = []
                        for j in answers:
                            answers_in_link.append(j.text)
                        code_answer_in_link.append(answers_in_link)
                        code_question_in_link.append(questions[0].text)
                        break
                    else:
                        pass
            count += 1
            if count%10 == 0:
                print(f"Thread{number_thread}:count : {count}")
                print("*"*20)
        code_answer[number_thread] = code_answer_in_link
        code_question[number_thread] = code_question_in_link
        print(f"thread : {number_thread} finish")
        print(time.time()-st,"sec")
        print("*"*20)
    except KeyError as err:
        print(f"thread : {number_thread} error\n{err}")

In [6]:
li = (page_last - 1)
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    if i < page_last:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(code_question,code_answer,i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i))
        print(i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i)
    elif page_last == 0:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(code_question,code_answer,i*num_per_thread,(i+1)*num_per_thread,i))
        print(i*num_per_thread,(i+1)*num_per_thread,i)
    else:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(code_question,code_answer,i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i))
        print(i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i)

0 3 0
3 6 1
6 9 2
9 12 3
12 15 4
15 18 5
18 21 6
21 24 7


In [7]:
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    all_thread[i].start()
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    all_thread[i].join()

thread : 5 finish
23.25523591041565 sec
********************
thread : 6 finish
26.651387214660645 sec
********************
thread : 2 finish
28.220577716827393 sec
********************
thread : 7 finish
28.66622233390808 sec
********************
thread : 4 finish
28.85326075553894 sec
********************
thread : 3 finish
30.159202814102173 sec
********************
thread : 1 finish
30.436578035354614 sec
********************
thread : 0 finish
36.004334926605225 sec
********************


In [14]:
code_answer[0][0][0]

'yield'

In [16]:
code_question[0][0]

'What does the "yield" keyword do in Python?'

In [17]:
with open(f"questions_data_python_stackoverflow_{vm_number}",'wb') as f:
    pickle.dump(code_question,f)

In [18]:
with open(f"answers_data_python_stackoverflow_{vm_number}",'wb') as f:
    pickle.dump(code_answer,f)

In [19]:
with open(f"questions_data_python_stackoverflow_{vm_number}",'rb') as f:
    ll = pickle.load(f)

In [26]:
ll[0][0]

'What does the "yield" keyword do in Python?'

In [22]:
with open(f"answers_data_python_stackoverflow_{vm_number}",'rb') as f:
    cc = pickle.load(f)

In [25]:
len(cc[0][0])

454